# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235216789903                   -0.50    8.0
  2   -7.250360497076       -1.82       -1.40    1.0
  3   -7.251316351630       -3.02       -2.11    4.0
  4   -7.251316148836   +   -6.69       -2.37    4.0
  5   -7.251334931768       -4.73       -2.80    3.0
  6   -7.251337661653       -5.56       -3.14    2.0
  7   -7.251338779666       -5.95       -4.06    1.0
  8   -7.251338791362       -7.93       -4.15    4.0
  9   -7.251338798057       -8.17       -4.68    2.0
 10   -7.251338798627       -9.24       -5.14    2.0
 11   -7.251338798700      -10.14       -5.75    3.0
 12   -7.251338798704      -11.43       -6.12    3.0
 13   -7.251338798704      -12.35       -6.49    2.0
 14   -7.251338798705      -12.93       -7.09    3.0
 15   -7.251338798705      -13.82       -7.69    3.0
 16   -7.251338798705   +  -14.45       -8.05    3.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04052489225650872
[ Info: Arnoldi iteration step 2: normres = 0.42567672447951194
[ Info: Arnoldi iteration step 3: normres = 0.7424133993171163
[ Info: Arnoldi iteration step 4: normres = 0.4631709479744236
[ Info: Arnoldi iteration step 5: normres = 0.5997386692139995
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.36e-01, 8.40e-02, 5.31e-01, 2.28e-01, 2.13e-02)
[ Info: Arnoldi iteration step 6: normres = 0.23680209334855223
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.66e-02, 1.16e-01, 1.57e-01, 9.78e-02, 3.71e-02)
[ Info: Arnoldi iteration step 7: normres = 0.11045732931163146
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.43e-03, 1.22e-02, 1.52e-02, 8.45e-02, 6.04e-02)
[ Info: Arnoldi iteration step 8: normres = 0.12048147691851274
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (7.61e